In [ ]:
%%capture --no-stdout
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: read).
The token `llm(Phi-3.5)=agricultura=20.11.24` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have 

In [ ]:
%%capture capturado_unsloth
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [ ]:
from unsloth import FastLanguageModel
import torch
configuracao_model = [524, None, True]
model_selecionado = "unsloth/Phi-3.5-mini-instruct"
model_llm, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_selecionado,
    max_seq_length=configuracao_model[0],
    dtype=configuracao_model[1],
    load_in_4bit=configuracao_model[2]
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.11.9: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/140 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.37k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

In [ ]:
model_llm = FastLanguageModel.get_peft_model(
    model_llm,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None
)

Unsloth 2024.11.9 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
from datasets import load_dataset
dataset = load_dataset("Solshine/Reflection-Tuning-Natural-Farming_Agricultural-Dataset", split="train")
print(dataset.column_names)
print(dataset[0])

README.md:   0%|          | 0.00/238 [00:00<?, ?B/s]

(…)a-combined-without-null-without-null.csv:   0%|          | 0.00/701k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/158 [00:00<?, ? examples/s]

['conversation_id', 'prompt', 'response']
{'conversation_id': 'Conversation 1', 'prompt': "I'm a soil scientist working with precision agriculture farmers in Paraná, Brazil. How can we use swales and bulb propagation to improve soil microbial activity for better hemp and peaches yields and increase farm biodiversity?", 'response': "<thinking>\nThe user is asking how to use swales and bulb propagation to improve soil microbial activity for better hemp and peach yields and increase farm biodiversity in Paraná, Brazil. \n\n<reflection>\nSwales are used for water management and soil conservation, which can enhance soil microbial activity by maintaining moisture levels. Bulb propagation can increase plant diversity and root structures, which also supports microbial life.\n\n</reflection>\n<reflection>\nThe user is interested in improving soil microbial activity, which suggests the use of Indigenous Microorganisms (IMOs) and Lactic Acid Bacteria (LABs) as per Korean Natural Farming principle

In [ ]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template="phi-3",  # Suporta vários formatos
    mapping={"role": "from", "content": "value", "user": "human", "assistant": "gpt"},  # ShareGPT style
)

# Função para formatar prompts e respostas
def formatting_prompts_func(examples):
    prompts = examples.get("prompt", [])
    responses = examples.get("response", [])
    texts = []

    for prompt, response in zip(prompts, responses):
        if prompt and response:
            try:
                user_text = tokenizer.apply_chat_template(
                    [{"role": "user", "content": prompt}],
                    tokenize=False,
                    add_generation_prompt=False,
                )
                assistant_text = tokenizer.apply_chat_template(
                    [{"role": "assistant", "content": response}],
                    tokenize=False,
                    add_generation_prompt=False,
                )
                texts.append(user_text + assistant_text)
            except Exception as e:
                print(f"Erro ao formatar: {e}")
                texts.append("")  # Garante que o comprimento seja consistente
        else:
            texts.append("")  # Garante que o comprimento seja consistente

    return {"text": texts}

# Aplicar a função de formatação para cada amostra
formatted_dataset = dataset.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/158 [00:00<?, ? examples/s]

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model=model_llm,
    tokenizer=tokenizer,
    train_dataset=formatted_dataset,
    dataset_text_field="text",
    max_seq_length=configuracao_model[0],  # Tamanho máximo da sequência
    dataset_num_proc=2,  # Número de processos para carregar o dataset
    args=TrainingArguments(
        per_device_train_batch_size=2,  # Tamanho menor de batch
        gradient_accumulation_steps=4,  # Maior acumulação de gradientes
        warmup_steps=5,  # Número de steps para warm-up
        max_steps=1000,  # Número máximo de steps de treinamento
        learning_rate=2e-4,  # Taxa de aprendizado
        fp16=not is_bfloat16_supported(),  # Usa FP16 se bfloat16 não for suportado
        bf16=is_bfloat16_supported(),  # Usa bfloat16 se suportado
        logging_steps=10,  # Logs de treinamento a cada step
        optim="adamw_8bit",  # Usar o otimizador AdamW com precisão de 8 bits
        weight_decay=0.01,  # Decaimento de peso
        lr_scheduler_type="linear",  # Tipo de agendador de taxa de aprendizado
        seed=3407,  # Semente para reprodutibilidade
        output_dir="outputs",  # Diretório para salvar os resultados
        report_to="none",  # Desativa o envio de relatórios para plataformas externas (ex: WandB)
    ),
)

Map (num_proc=2):   0%|          | 0/158 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
2.285 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()
model_llm.save_pretrained("./loremel")  # Salvar o modelo
tokenizer.save_pretrained("./loremel")  # Salvar o tokenizer


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 158 | Num Epochs = 53
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 1,000
 "-____-"     Number of trainable parameters = 29,884,416


Step,Training Loss
10,1.176800
20,0.933600
30,0.737400
40,0.645200
50,0.536200
60,0.496900
70,0.387300
80,0.332900
90,0.259600
100,0.216600


In [ ]:
from unsloth.chat_templates import get_chat_template
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Carregue o modelo e o tokenizer
model_baixado = GPT2LMHeadModel.from_pretrained("./loremel")
tokenizer_baixado = GPT2Tokenizer.from_pretrained("./loremel")

# Ajuste do template de chat
tokenizer_baixado = get_chat_template(
    tokenizer_baixado,
    chat_template="phi-3",
    mapping={"role": "from", "content": "value", "user": "human", "assistant": "gpt"},
)

# Ativar inferência mais rápida e reduzir uso de memória usando FP16, se possível
if torch.cuda.is_available():
    model_baixado = model_baixado.half().to('cuda')  # Usar FP16 e mover para GPU

# Função de chat interativo
def chat_with_model():
    print("Comece a conversar com o modelo! (Digite 'sair' para finalizar)")

    conversation = []
    while True:
        user_input = input("Você: ")  # Input do usuário

        # Se o usuário digitar 'sair', termina o chat
        if user_input.lower() == 'sair':
            print("Encerrando o chat.")
            break

        # Adiciona a mensagem do usuário à conversa
        conversation.append({"from": "human", "value": user_input})

        # Prepara entradas para o modelo
        inputs = tokenizer_baixado.apply_chat_template(
            conversation,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt",
        ).to("cuda" if torch.cuda.is_available() else "cpu")

        # Gera a resposta do modelo
        outputs = model_baixado.generate(input_ids=inputs['input_ids'], max_new_tokens=32, use_cache=True)

        # Decodifica e exibe a resposta do modelo
        response = tokenizer_baixado.batch_decode(outputs, skip_special_tokens=True)[0]
        print(f"Modelo: {response}")

        # Adiciona a resposta do modelo à conversa para continuar o diálogo
        conversation.append({"from": "gpt", "value": response})

# Inicia o chat
chat_with_model()

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


You are using a model of type llama to instantiate a model of type gpt2. This is not supported for all configurations of models and can yield errors.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


# **Multiplico treinamento**

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("./loremel")
tokenizer = AutoTokenizer.from_pretrained("./loremel")


`low_cpu_mem_usage` was None, now default to True since model is quantized.


In [ ]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=formatted_dataset,
    dataset_text_field="text",
    max_seq_length=configuracao_model[0],
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        num_train_epochs=300,
        save_steps=50,
        save_total_limit=10,
        learning_rate=2e-4,
        # Change fp16 to False to disable mixed precision training using bf16
        fp16=False,
        logging_steps=10,
        # Change optim to "adamw_hf" to avoid bf16 operations
        optim="adamw_hf",  # Changed from "adamw_8bit"
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        overwrite_output_dir=True,
        report_to="none",
    ),
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 158 | Num Epochs = 300
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 5,700
 "-____-"     Number of trainable parameters = 0


RuntimeError: Internal Triton PTX codegen error: 
ptxas /tmp/tmpv5mfs442.ptx, line 107; error   : Feature '.bf16' requires .target sm_80 or higher
ptxas /tmp/tmpv5mfs442.ptx, line 107; error   : Feature 'cvt with .f32.bf16' requires .target sm_80 or higher
ptxas /tmp/tmpv5mfs442.ptx, line 111; error   : Feature '.bf16' requires .target sm_80 or higher
ptxas /tmp/tmpv5mfs442.ptx, line 111; error   : Feature 'cvt with .f32.bf16' requires .target sm_80 or higher
ptxas /tmp/tmpv5mfs442.ptx, line 115; error   : Feature '.bf16' requires .target sm_80 or higher
ptxas /tmp/tmpv5mfs442.ptx, line 115; error   : Feature 'cvt with .f32.bf16' requires .target sm_80 or higher
ptxas /tmp/tmpv5mfs442.ptx, line 119; error   : Feature '.bf16' requires .target sm_80 or higher
ptxas /tmp/tmpv5mfs442.ptx, line 119; error   : Feature 'cvt with .f32.bf16' requires .target sm_80 or higher
ptxas /tmp/tmpv5mfs442.ptx, line 123; error   : Feature '.bf16' requires .target sm_80 or higher
ptxas /tmp/tmpv5mfs442.pt

In [ ]:
%%capture capturado_unsloth
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git
from unsloth import FastLanguageModel
import torch
configuracao_model = [1024, None, True]
model_selecionado = "unsloth/Phi-3.5-mini-instruct"
model_llm, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_selecionado,
    max_seq_length=configuracao_model[0],
    dtype=configuracao_model[1],
    load_in_4bit=configuracao_model[2]
)
model_llm = FastLanguageModel.get_peft_model(
    model_llm,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None
)
dataset = load_dataset("Solshine/Reflection-Tuning-Natural-Farming_Agricultural-Dataset", split="train")
print(dataset.column_names)
print(dataset[0])
from datasets import load_dataset
from unsloth.chat_templates import get_chat_template

# Configuração do tokenizer com o template de chat
tokenizer = get_chat_template(
    tokenizer,
    chat_template="phi-3",  # Suporta vários formatos
    mapping={"role": "from", "content": "value", "user": "human", "assistant": "gpt"},  # ShareGPT style
)

# Função para formatar os prompts e respostas
def formatting_prompts_func(examples):
    prompts = examples.get("prompt", [])
    responses = examples.get("response", [])
    texts = []

    for prompt, response in zip(prompts, responses):
        if prompt and response:
            try:
                user_text = tokenizer.apply_chat_template(
                    [{"role": "user", "content": prompt}],
                    tokenize=False,
                    add_generation_prompt=False,
                )
                assistant_text = tokenizer.apply_chat_template(
                    [{"role": "assistant", "content": response}],
                    tokenize=False,
                    add_generation_prompt=False,
                )
                texts.append(user_text + assistant_text)
            except Exception as e:
                print(f"Erro ao formatar: {e}")
                texts.append("")  # Garante que o comprimento seja consistente
        else:
            texts.append("")  # Garante que o comprimento seja consistente

    return {"text": texts}

# Carregar o dataset
dataset = load_dataset("Solshine/Reflection-Tuning-Natural-Farming_Agricultural-Dataset", split="train")

# Aplicar a função de formatação para cada amostra
formatted_dataset = dataset.map(formatting_prompts_func, batched=True)

from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

# Certifique-se de que a variável 'configuracao_model' contém o valor adequado para max_seq_length
max_seq_length = configuracao_model[0]  # Isso depende da configuração do modelo que você está usando

trainer = SFTTrainer(
    model=model_llm,  # Seu modelo LLM a ser treinado
    tokenizer=tokenizer,  # O tokenizer correspondente
    train_dataset=formatted_dataset,  # Dataset formatado
    dataset_text_field="text",  # Campo de texto no dataset, se ajustado para "text"
    max_seq_length=max_seq_length,  # Tamanho máximo da sequência (definido pela configuração)
    dataset_num_proc=2,  # Número de processos para carregar o dataset
    packing=False,  # Se for False, não utiliza packing de sequências curtas (mantém mais controle)
    args=TrainingArguments(
        per_device_train_batch_size=2,  # Tamanho do batch por dispositivo
        gradient_accumulation_steps=4,  # Passos de acumulação de gradientes para simular batches maiores
        warmup_steps=5,  # Número de steps para warm-up
        max_steps=100,  # Número máximo de steps de treinamento
        learning_rate=2e-4,  # Taxa de aprendizado
        fp16=not is_bfloat16_supported(),  # Usa FP16 se bfloat16 não for suportado
        bf16=is_bfloat16_supported(),  # Usa bfloat16 se suportado
        logging_steps=1,  # Logs de treinamento a cada step
        optim="adamw_8bit",  # Usar o otimizador AdamW com precisão de 8 bits para economizar memória
        weight_decay=0.01,  # Decaimento de peso para evitar overfitting
        lr_scheduler_type="linear",  # Tipo de agendador de taxa de aprendizado
        seed=3407,  # Semente para reprodutibilidade
        output_dir="outputs",  # Diretório para salvar os resultados
        report_to="none",  # Desativa o envio de relatórios para plataformas externas (ex: WandB)
    ),
)

#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")
trainer_stats = trainer.train()
model_llm.save_pretrained("./loremel") # Local saving
tokenizer.save_pretrained("./loremel")

from unsloth.chat_templates import get_chat_template
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# Carregue o modelo e o tokenizer
model_baixado = GPT2LMHeadModel.from_pretrained("./loremel")
tokenizer_baixado = GPT2Tokenizer.from_pretrained("./loremel")

# Ajuste do template de chat
tokenizer_baixado = get_chat_template(
    tokenizer_baixado,
    chat_template="phi-3",
    mapping={"role": "from", "content": "value", "user": "human", "assistant": "gpt"},
)

# Ativar inferência mais rápida e reduzir uso de memória usando FP16, se possível
if torch.cuda.is_available():
    model_baixado = model_baixado.half().to('cuda')  # Usar FP16 e mover para GPU

# Função de chat interativo
def chat_with_model():
    print("Comece a conversar com o modelo! (Digite 'sair' para finalizar)")

    conversation = []
    while True:
        user_input = input("Você: ")  # Input do usuário

        # Se o usuário digitar 'sair', termina o chat
        if user_input.lower() == 'sair':
            print("Encerrando o chat.")
            break

        # Adiciona a mensagem do usuário à conversa
        conversation.append({"from": "human", "value": user_input})

        # Prepara entradas para o modelo
        inputs = tokenizer_baixado.apply_chat_template(
            conversation,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt",
        ).to("cuda" if torch.cuda.is_available() else "cpu")  # Verifica se tem CUDA disponível

        # Gera a resposta do modelo
        outputs = model_baixado.generate(input_ids=inputs['input_ids'], max_new_tokens=32, use_cache=True)

        # Decodifica e exibe a resposta do modelo
        response = tokenizer_baixado.batch_decode(outputs, skip_special_tokens=True)[0]
        print(f"Modelo: {response}")

        # Adiciona a resposta do modelo à conversa para continuar o diálogo
        conversation.append({"from": "gpt", "value": response})

# Inicia o chat
chat_with_model()
